<a href="https://colab.research.google.com/github/Arnav-Ajay/Article-Parsing-System/blob/main/Article_Parsing_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Article Parsing System for Neural Network Processing

A system for collecting articles and storing them in a database that will be compatible with neural network processing.

The system uses JSON for article storage and SQLite as the database.

**Initial Source:** https://cointelegraph.com/  or  https://cointelegraph.com/rss

**Database Fields:**
- Title
- Author
- Publication date
- Article text
- Article URL
- Outbound links within the article
- Internal links (links to other articles on the same website)
- Tags

In [1]:
! pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.9 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=a77e962036924b55e7c50ae6b8212ec28682d009d3caa2f8ae9e066d3be52ce3
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [2]:
import pandas as pd
import feedparser
import sqlite3
import json
import os

## Functions

#### RSS Feed

Note: https://cointelegraph.com/rss is restricted. we need permission to extract each article. "Article text", "Outbound Links", and "Inbound Links" are incomplete because of this.

Might want to consider a different site or asking them for permission.

Check https://cointelegraph.com/robots.txt for their policy.

In [3]:
# get data from a RSS feed - Modified to incorporate Cointelegraph RSS feed
def fetch_rss_feed(url):
  """
  Fetches and parses an RSS feed from the given URL

  Args:
    url (str): The URL of the RSS feed.

  Returns:
    dict: A dictionary containing the feed data.
  """
  try:
    # Parse the RSS feed
    feed = feedparser.parse(url)
    if feed.bozo:
      # If there was a problem with the feed
      print(f"Error reading feed: {feed.bozo_exception}")
      return {}

    # Extract all feed data
    feed_data = {
        "feed": {
            "title": feed.feed.get("title", "No Title"),
            "link": feed.feed.get("link", "No Link"),
            "description": feed.feed.get("description", "No Description"),
            "updated": feed.feed.get("updated", "No Update Time"),
          },
          "entries": []
    }
    feed_host = feed.feed.link if "link" in feed.feed else url

    for entry in feed.entries:
      # Initialize sets for unique links
      inbound_links = set()
      outbound_links = set()

      # looping over each entry to retrieve outbound and inbound links
      entry_link = entry.get("link", "")
      if entry_link.startswith(feed_host):
        inbound_links.add(entry_link)
      else:
        outbound_links.add(entry_link)
      # Extractive only the tags of each entry
      tags = entry.get("tags", "No Tags") if "tags" in entry else "No Tags",
      str_tags = [item.term for item in tags[0]]
      # Extract all Entries
      feed_data["entries"].append({
          # Source (media outlet name)
          "title": entry.get("title", "No Title"),
          "author": entry.get("author", "No Author"),
          "article_text": entry.get("description", "No Description"),
          "publication_date": entry.get("published", "No Published Date"),
          "article_link": entry.get("link", "No Link"),
          "tags": ', '.join(str(x) for x in str_tags),
          "inbound_links" : ', '.join(str(x) for x in inbound_links),
          "outbound_links" : ', '.join(str(x) for x in outbound_links),
      })
    return feed_data

  except Exception as e:
    print(f"An error occurred: {e}")
    return {}

#### SQLite Database

In [4]:
def create_table_from_json(cursor, table_name, json_data):
    """
    Create a table in SQLite based on JSON data structure.

    Args:
        cursor: SQLite cursor object.
        table_name (str): Name of the table to create.
        json_data (list): List of dictionaries representing the JSON data.
    """
    if not json_data:
        raise ValueError(f"JSON data is empty or invalid in file {json_data}")

    # Extract column names and types from the first record in the JSON data
    columns = json_data["entries"][0].keys()

    # Define a SQLite CREATE TABLE query with dynamic columns
    column_definitions = ", ".join([f"{col} TEXT" for col in columns])
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({column_definitions})"

    # Execute the CREATE TABLE query
    cursor.execute(create_table_query)

In [5]:
def insert_data_into_table(cursor, table_name, json_data):
    """
    Insert JSON data into an SQLite table.

    Args:
        cursor: SQLite cursor object.
        table_name (str): Name of the table to insert data into.
        json_data (list): List of dictionaries representing the JSON data.
    """
    if not json_data:
        raise ValueError("JSON data is empty or invalid - No Data to insert")

    # Extract column names
    columns = json_data[0].keys()
    column_names = ", ".join(columns)
    placeholders = ", ".join(["?"] * len(columns))

    # Prepare the INSERT query
    insert_query = f"INSERT INTO {table_name} ({column_names}) VALUES ({placeholders})"

    # Insert each record
    for record in json_data:
        values = tuple(record.values())
        cursor.execute(insert_query, values)

In [6]:
def validate_new_entries(cursor, table_name, json_data):
  new_data = {}

  # Fetch all data from the table
  query = f"SELECT * FROM {table_name}"
  cursor.execute(query)

  rows = [row for row in cursor.fetchall()]
  if len(rows) == 0:
    new_data = json_data['entries']
    print(f"New Entries: {len(new_data)}")
    return new_data

  # Extract existing keys from db for quick lookup
  existing_keys = {entry['title'] for entry in json_data['entries'] if 'title' in entry}

  # Add new entries from json_data to new_data if their unique_key value is not in db_data
  for entry in json_data['entries']:
    if entry.get('title') not in existing_keys:
      new_data.update(entry)
  print(f"New Entries: {len(new_data)}")
  return new_data

In [7]:
def load_json_to_sqlite(json_file, db_file, table_name):
    """
    Load data from a JSON file into an SQLite database.

    Args:
        json_file (str): Path to the JSON file.
        db_file (str): Path to the SQLite database file.
        table_name (str): Name of the table to create/insert data into.
    """
    try:

        # Connect to SQLite database
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Load JSON data
        with open(json_file, "r") as f:
            json_data = json.load(f)

        # Create the table and insert data
        create_table_from_json(cursor, table_name, json_data)

        json_data = validate_new_entries(cursor, table_name, json_data)
        if json_data:
          insert_data_into_table(cursor, table_name, json_data)

        conn.commit()

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close the database connection
        if conn:
            conn.close()

In [8]:
def read_data_from_table(db_file, table_name):
    """
    Reads all data from a specified SQLite table.

    Args:
        db_file (str): Path to the SQLite database file.
        table_name (str): Name of the table to read data from.

    Returns:
        list: A list of rows from the table as dictionaries.
    """
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_file)
        # conn.row_factory = sqlite3.Row
        cursor = conn.cursor()

        # Fetch all data from the table
        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)

        headers = [description[0] for description in cursor.description]
        # Convert rows to a list of dictionaries
        rows = [row for row in cursor.fetchall()]
        return [dict(zip(headers, row)) for row in rows]

    except Exception as e:
        print(f"Error: {e}")
        return []

    finally:
        # Close the database connection
        if conn:
            conn.close()

## Main

In [9]:
# Global Variables and fetching RSS Feed
url = "https://cointelegraph.com/rss"
json_file_path = "/content/feed_data.json"
sqlite_db_path = "/content/data.db"
table_name = "example_table"

data = fetch_rss_feed(url)

In [10]:
# This block has not been validated yet.
# Saving the feed to json file,
# Appending json file if already exists
if os.path.exists(json_file_path):
  with open(json_file_path, "r", encoding="utf-8") as json_file:
    existing_data = json.load(json_file)
  print(len(existing_data['entries']))

  existing_keys = {entry['title'] for entry in existing_data['entries'] if 'title' in entry}
  # Add new entries from json_data to new_data if their unique_key value is not in db_data
  for entry in data['entries']:
    if entry.get('title') not in existing_keys:
      existing_data['entries'].update(entry)
  print(len(existing_data['entries']))

  with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(existing_data, json_file, indent=4, ensure_ascii=False)

else:
  with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, indent=4, ensure_ascii=False)

print(f"Feed data successfully saved to {json_file_path}")

Feed data successfully saved to /content/feed_data.json


In [11]:
# Saving to DB
load_json_to_sqlite(json_file_path, sqlite_db_path, table_name)

# Fetch all data from the table
data = read_data_from_table(sqlite_db_path, table_name)
df_data = pd.DataFrame(data)
df_data.head()

New Entries: 30


,title,author,article_text,publication_date,article_link,tags,inbound_links,outbound_links
0,Crypto products record 10th consecutive week o...,Cointelegraph by Helen Partz,"<p style=""float: right; margin: 0 0 10px 15px;...","Mon, 16 Dec 2024 11:26:59 +0000",https://cointelegraph.com/news/crypto-etp-3-2-...,"Bitcoin ETF, Ethereum ETF, CoinShares, Investm...",https://cointelegraph.com/news/crypto-etp-3-2-...,
1,Why is the crypto market up today?,Cointelegraph by Nancy Lubale,"<p style=""float: right; margin: 0 0 10px 15px;...","Mon, 16 Dec 2024 10:48:15 +0000",https://cointelegraph.com/news/why-is-the-cryp...,"Bitcoin, BTC price, crypto, SEC, BlackRock, wh...",https://cointelegraph.com/news/why-is-the-cryp...,
2,Intrinsic value of crypto: What is it and how ...,Cointelegraph by Onkar Singh,"<p style=""float: right; margin: 0 0 10px 15px;...","Mon, 16 Dec 2024 10:39:00 +0000",https://cointelegraph.com/news/intrinsic-value...,Intrinsic value of crypto,https://cointelegraph.com/news/intrinsic-value...,
3,Bitcoin to gold ratio posts new record as BTC ...,Cointelegraph by Helen Partz,"<p style=""float: right; margin: 0 0 10px 15px;...","Mon, 16 Dec 2024 09:22:37 +0000",https://cointelegraph.com/news/bitcoin-to-gold...,"Gold, Bitcoin, Bitcoin-to-gold ratio, Dollar, ...",https://cointelegraph.com/news/bitcoin-to-gold...,
4,BTC price &#039;base case&#039; now $140K: 5 t...,Cointelegraph by William Suberg,"<p style=""float: right; margin: 0 0 10px 15px;...","Mon, 16 Dec 2024 09:06:45 +0000",https://cointelegraph.com/news/btc-price-base-...,"Bitcoin, BTC price",https://cointelegraph.com/news/btc-price-base-...,
